## Import

First we need to import some popular python libraries.

In [1]:
import csv
import errno
import glob
import os
import re
import string
import sys

from collections import OrderedDict, Counter
from subprocess import check_call
from shutil import copyfile

from tqdm import tqdm

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import xgboost as xgb

In [3]:
from sklearn import decomposition, ensemble, metrics, model_selection, naive_bayes, preprocessing, pipeline
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import SGDClassifier as sgd
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tag import StanfordNERTagger

C:\ProgramData\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [5]:
from keras import initializers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import GlobalAveragePooling1D,Merge,Lambda,Input,GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D,TimeDistributed
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence, text
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils

Using TensorFlow backend.


In [6]:
# from sner import Ner 

## Setting

We let IPython to show all the variables automatically rather than only the last one as default.

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Then we mute the `pandas` when we try to use chained assignments. Check this for reference: <https://www.dataquest.io/blog/settingwithcopywarning/>

In [8]:
pd.options.mode.chained_assignment = None

## Read CSV

Here is how we use pandas to read csv files to get the training data and test data.

In [9]:
train_path = '../input/train.csv'
test_path = '../input/test.csv'


## Read the train and test dataset and check the top few lines ##
# TODO:
train_df = pd.read_csv(train_path).fillna('NANN').head(5000)
test_df = pd.read_csv(test_path).fillna('NANN')

train_df.shape[0]
test_df.shape[0]
train_df.head()
test_df.head()

5000

226998

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


,id,comment_text
0,6044863,==Orphaned non-free media (Image:41cD1jboEvL. ...
1,6102620,::Kentuckiana is colloquial. Even though the ...
2,14563293,"Hello fellow Wikipedians,\nI have just modifie..."
3,21086297,"AKC Suspensions \nThe Morning Call - Feb 24, 2..."
4,22982444,== [WIKI_LINK: Talk:Celts] ==


We can also access the target labels and select the content of particular labels:

In [10]:
labels = list(train_df)[2:]
len(labels)
labels
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_df[labels].head()

label0 = labels[0]
train_df[label0].head()

train_df[train_df['toxic']==1]['comment_text'].head()

6

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0


0    1
1    0
2    0
3    0
4    0
Name: toxic, dtype: int64

0     Nonsense?  kiss off, geek. what I said is true...
20    Why hasn't Alitalia been removed rom the allia...
26    "\nThe Graceful Slick....\nIs non other than a...
30    "\n\n Stupid? \n\nAs soon as I saw the phrase ...
32    "\nBan one side of an argument by a bullshit n...
Name: comment_text, dtype: object

Then we store the `id` field of test file for submission, then drop it from train and test data frame for simplicity. We also get the target labels as `train_y`, and drop them from train data frame.

In [11]:
## Prepare the data for modeling ###
train_y = train_df[labels].values
test_id = test_df['id'].values
labels
train_y

cols_to_drop = ['id']
train_df = train_df.drop(cols_to_drop + labels, axis=1)
test_df = test_df.drop(cols_to_drop, axis=1)

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

## Tokenize

In [12]:
for df in [train_df, test_df]:
    df['split'] = df['comment_text'].apply(word_tokenize)
    print('split finished...')

split finished...
split finished...


## Named Entity

In [13]:
def feature_ner(df):
    st = StanfordNERTagger('../stanford-ner-2017-06-09/classifiers/english.all.3class.distsim.crf.ser.gz',
                          '../stanford-ner-2017-06-09/stanford-ner.jar',
                          encoding='utf-8')
    st = Ner(host='localhost',port=9199)
    df['st'] = df['comment_text'].apply(lambda x: [w[1] for w in st.get_entities(x)])
    df['n_person'] = df['st'].apply(lambda x: x.count('PERSON'))
    df['n_location'] = df['st'].apply(lambda x: x.count('LOCATION'))
    print('NER finished...')

## Sentiment Intensity

In [14]:
def feature_sid(df):
    sid = SentimentIntensityAnalyzer()
    df['sid'] = df['comment_text'].apply(sid.polarity_scores)
    for k in ['neu', 'compound', 'pos', 'neg']:
        df['sid_'+k] = df['sid'].apply(lambda x: x[k])
    print('polarity_scores finished...') 

## Part of Speech

In [15]:
def feature_pos(df): 
    df['pos_tag'] = df['split'].apply(lambda x: [w[1] for w in nltk.pos_tag(x)])
    for pos in ['CC', 'RB', 'IN', 'NN', 'VB', 'VBP', 'JJ', 'PRP', 'TO', 'DT']:
        df['n_pos_' + pos] = df['pos_tag'].apply(lambda x: x.count(pos))
    print('pos_tag finished...')

In [16]:
def feature_cnt(df):
    ## Number of words in the text ##
    df["num_words"] = df["split"].apply(len)
    
    return
    ## Number of unique words in the text ##
    df["num_unique_words"] = df["split"].apply(lambda x: len(set(x)))
    
    ## Number of characters in the text ##
    df["num_chars"] = df['comment_text'].apply(len)
    
    ## Number of stopwords in the text ##
    #eng_stopwords = set(stopwords.words("english"))
    df["num_stopwords"] = df["split"].apply(lambda x: len([w for w in x if w in eng_stopwords]))
    
    ## Number of punctuations in the text ##
    df["num_punctuations"] = df['split'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
    
    ## Number of title case words in the text ##
    df["num_words_upper"] = df["split"].apply(lambda x: len([w for w in x if w.isupper()]))
    
    ## Number of title case words in the text ##
    df["num_words_title"] = df["split"].apply(lambda x: len([w for w in x if w.istitle()]))
    
    ## Average length of the words in the text ##
    df["mean_word_len"] = df["split"].apply(lambda x: np.mean([len(w) for w in x]))


    anchor_words = ['the', 'a', 'appear', 'little', 'was', 'one', 'two', 'three', 'ten', 'is', 
                    'are', 'ed', 'however', 'to', 'into', 'about', 'th', 'er', 'ex', 'an', 
                    'ground', 'any', 'silence', 'wall']

    gender_words = ['man', 'woman', 'he', 'she', 'her', 'him', 'male', 'female']

    for word in anchor_words + gender_words:
        df['n_'+word] = df["split"].apply(lambda x: len([w for w in x if w.lower() == word]))

In [17]:
%%time
for df in [train_df, test_df]:
    feature_cnt(df)

Wall time: 117 ms


In [18]:
list(train_df)

['comment_text', 'split', 'num_words']

## Topic Model

In [19]:
def feature_lda(train_df, test_df): 
    
    ### Fit transform the tfidf vectorizer ###
    tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
    full_tfidf = tfidf_vec.fit_transform(train_df['comment_text'].values.tolist() + test_df['comment_text'].values.tolist())
    train_tfidf = tfidf_vec.transform(train_df['comment_text'].values.tolist())
    test_tfidf = tfidf_vec.transform(test_df['comment_text'].values.tolist())
    
    no_topics = 20 
    lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(full_tfidf)
    train_lda = pd.DataFrame(lda.transform(train_tfidf))
    test_lda = pd.DataFrame(lda.transform(test_tfidf))
    
    train_lda.columns = ['lda_'+str(i) for i in range(no_topics)]
    test_lda.columns = ['lda_'+str(i) for i in range(no_topics)]
    train_df = pd.concat([train_df, train_lda], axis=1)
    test_df = pd.concat([test_df, test_lda], axis=1)
    del full_tfidf, train_tfidf, test_tfidf, train_lda, test_lda

    print("LDA finished...")


## Neural Network

### Glove Vector

In [20]:
# load the GloVe vectors in a dictionary:

wv = '../input/glove.6B.100d.txt'

def loadWordVecs():
    embeddings_index = {}
    f = open(wv)
    for line in f:
        print(line)
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    return embeddings_index

def sent2vec(embeddings_index,s): # this function creates a normalized vector for the whole sentence
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stopwords.words('english')]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(100)
    return v / np.sqrt((v ** 2).sum())

def doGlove(x_train,x_test):
    embeddings_index = loadWordVecs()
    # create sentence vectors using the above function for training and validation set
    xtrain_glove = [sent2vec(embeddings_index,x) for x in tqdm(x_train)]
    xtest_glove = [sent2vec(embeddings_index,x) for x in tqdm(x_test)]
    xtrain_glove = np.array(xtrain_glove)
    xtest_glove = np.array(xtest_glove)
    return xtrain_glove,xtest_glove,embeddings_index

glove_vecs_train,glove_vecs_test,embeddings_index = doGlove(train_df['comment_text'], test_df['comment_text'])
train_df[['sent_vec_'+str(i) for i in range(100)]] = pd.DataFrame(glove_vecs_train.tolist())
test_df[['sent_vec_'+str(i) for i in range(100)]] = pd.DataFrame(glove_vecs_test.tolist())
print("Glove sentence vector finished...")

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062

, -0.10767 0.11053 0.59812 -0.54361 0.67396 0.10663 0.038867 0.35481 0.06351 -0.094189 0.15786 -0.81665 0.14172 0.21939 0.58505 -0.52158


for -0.14401 0.32554 0.14257 -0.099227 0.72536 0.19321 -0.24188 0.20223 -0.89599 0.15215 0.035963 -0.59513 -0.051635 -0.014428 0.35475 -0.31859 0.76984 -0.087369 -0.24762 0.65059 -0.15138 -0.42703 0.18813 0.091562 0.15192 0.11303 -0.15222 -0.62786 -0.23923 0.096009 -0.46147 0.41526 -0.30475 0.1371 0.16758 0.53301 -0.043658 0.85924 -0.41192 -0.21394 -0.51228 -0.31945 0.12662 -0.3151 0.0031429 0.27129 0.17328 -1.3159 -0.42414 -0.69126 0.019017 -0.13375 -0.096057 1.7069 -0.65291 -2.6111 0.26518 -0.61178 2.095 0.38148 -0.55823 0.2036 -0.33704 0.37354 0.6951 -0.001637 0.81885 0.51793 0.27746 -0.37177 -0.43345 -0.42732 -0.54912 -0.30715 0.18101 0.2709 -0.29266 0.30834 -1.4624 -0.18999 0.92277 -0.099217 -0.25165 0.49197 -1.525 0.15326 0.2827 0.12102 -0.36766 -0.61275 -0.18884 0.10907 0.12315 0.090066 -0.65447 -0.17252 2.6336e-05 0.25398 1.1078 -0.073074

- -1.2557 0.61036 0.56793 -0.96596 -0.45249 -0.071696 0.57122 -0.31292 -0.43814 0.90622 0.06961 -0.053104 0.25029 0.27841 0.77724 0.26329 0


it -0.30664 0.16821 0.98511 -0.33606 -0.2416 0.16186 -0.053496 0.4301 0.57342 -0.071569 0.36101 0.26729 0.27789 -0.072268 0.13838 -0.26714 0.12999 0.22949 -0.18311 0.50163 0.44921 -0.020821 0.42642 -0.068762 0.40337 0.095198 -0.31944 -0.54651 -0.13345 -0.56511 -0.20975 1.1592 -0.194 0.19828 -0.11924 0.41781 0.0068383 -0.20537 -0.53375 -0.52225 -0.38227 -0.0065833 0.14265 -0.42502 -0.3115 0.0027352 0.75093 -0.48218 -0.18595 -0.77104 -0.046406 -0.06914 0.41688 1.3235 -0.81742 -3.3998 -0.11307 -0.34123 2.0775 0.61369 0.14792 0.93753 -0.10138 0.28426 0.97899 -0.32335 0.63697 0.58308 0.2282 -0.31696 0.21061 -0.6506 0.21653 -0.24347 0.55519 -0.34351 -0.095093 -0.14715 -1.2876 0.3931 0.30163 -0.21767 -1.1146 0.51349 -1.341 -0.30381 0.32499 -0.45236 -0.17743 -0.048504 -0.12178 -0.42108 -0.40327 0.038452 -0.36084 0.037738 -0.21885 -0.38775 0.36916 0.54521

by -0.20875 -0.1174 0.26478 -0.28339 0.19584 0.7446 -0.03887 0.028499 -0.44252 -0.30426 0.27133 -0.51907 0.52183 -0.76648 0.28043 -0.48344 


be -0.46953 0.38432 0.54833 -0.63401 0.010133 0.11364 0.10612 0.58529 0.032302 -0.12274 0.030265 0.52662 1.0398 -0.082143 0.19118 -0.83784 0.50763 0.44488 -0.72604 0.036893 0.24211 -0.28878 0.33657 0.13656 0.14579 -0.13221 0.098428 -0.45276 -0.13029 0.015762 -0.010161 0.4967 -0.28461 0.29655 0.92979 0.42447 -0.082773 0.30438 -0.39219 -0.30585 -0.43201 -0.27333 0.24388 -0.58081 0.22679 0.027226 0.53473 -0.37527 -0.16119 -1.1235 0.12768 -0.69898 0.41341 1.2291 -0.41248 -2.5173 -0.15354 -0.043107 1.9111 0.80754 -0.14759 0.9609 -0.84267 0.084422 1.2616 -0.10938 0.54846 0.75255 -0.071289 -0.73987 0.094808 -0.97589 0.0078721 -0.23928 0.2882 -0.41516 0.034366 0.1197 -1.2142 -0.11306 0.52847 -0.42273 -0.93378 -0.046645 -2.122 -0.341 0.64229 -0.10097 -0.22875 -1.0776 -0.68044 -0.26372 -0.18331 -0.051632 -0.30836 0.066537 0.20422 -0.68914 0.4511 0.25125

has 0.093736 0.56152 0.48364 -0.45987 0.56067 -0.1694 0.018687 0.45529 0.065615 0.25181 -0.14251 0.10532 0.77865 0.1428 -0.08114 -0.069555 0.3


had 0.63256 -0.12718 -0.084182 -0.30718 -0.2526 -0.16172 0.47123 0.46553 -0.051526 0.17231 0.42743 0.57854 0.64548 0.31367 0.32752 -0.47608 0.25888 0.035845 -0.95154 -0.20671 0.65171 0.010712 0.3894 0.069552 0.061198 -0.72152 -0.22334 -0.34747 -0.1434 -0.32482 0.79539 0.84708 -0.046052 0.74384 0.18185 0.18666 0.17123 0.93485 -0.1299 0.39219 -1.016 -0.27859 0.79293 -0.40433 0.41505 0.017283 0.2936 -0.72944 0.98233 -0.96504 -0.19016 -0.010012 0.26765 1.3316 -0.19376 -2.4401 -0.39477 -0.36232 0.88315 1.5974 0.31113 0.72201 -0.55023 0.54995 0.6846 -0.40378 0.52677 0.23314 -0.12764 0.12305 -0.12489 -0.25052 0.11632 -0.41648 -0.19738 0.093786 -0.24532 0.29676 -1.6156 0.38933 0.68466 0.048371 -0.4177 0.10663 -2.0738 -0.81679 0.56651 -0.32536 -0.76685 -0.27123 -0.10798 -0.61763 -0.27098 0.077119 -0.89352 0.17811 -0.50156 -0.30966 0.22378 0.038183

i -0.046539 0.61966 0.56647 -0.46584 -1.189 0.44599 0.066035 0.3191 0.14679 -0.22119 0.79239 0.29905 0.16073 0.025324 0.18678 -0.31001 -0.28108 0.6

...


another -0.13669 0.16266 0.32851 -0.23838 0.37632 0.512 0.43825 0.2659 0.10699 0.10075 0.099575 0.23082 0.030345 -0.33396 0.38298 -0.24366 -0.011523 -0.35039 -0.20009 0.46843 0.79308 -0.30159 0.032797 0.28515 0.5024 -0.27093 -0.42409 -0.41382 0.094871 0.072463 -0.55992 0.4625 0.27367 -0.11924 -0.055437 -0.053994 -0.22305 0.19366 0.42554 0.12814 -0.30268 0.11386 0.66691 -0.56826 0.49106 0.35975 0.41939 0.055777 -0.52086 -0.44886 0.058267 -0.082595 0.082818 1.148 -0.44502 -3.0693 -0.38092 -0.22423 1.6572 0.69771 0.057998 0.87766 -0.032617 0.24173 0.72651 -0.24736 0.50189 0.58423 -0.22143 0.18851 -0.46677 -0.022507 -0.21069 -0.086704 0.31232 0.27984 -0.21408 -0.41711 -1.1358 0.018686 0.89265 -0.00067525 -0.26069 -0.14054 -1.1586 -0.32087 0.4694 0.19181 0.1376 -0.55373 0.42719 0.14677 -0.18532 -0.1694 -0.27725 -0.29067 0.17023 -0.1319 0.48349 0.011721

tuesday 0.035314 -0.45705 0.435 0.042954 0.16798 -0.082904 -0.00059971 0.77181 -0.57037 0.042372 -0.4633 0.11997 0.017448 -0.47586 -0.0806


used -0.4713 0.57094 -0.50343 -0.16902 0.207 0.20779 0.087041 0.049987 -0.14483 0.26928 -0.19927 -0.6827 0.27491 0.82178 0.31601 -0.11952 1.043 0.24766 -0.34924 -0.30117 0.40915 -0.34067 0.3716 0.093613 -0.13949 -0.5461 -0.3396 -0.2273 -0.05824 0.11287 0.15304 0.86929 -0.97688 0.22133 0.6068 0.56773 0.0032481 0.19031 0.48249 -0.10228 -0.14569 -0.36808 -0.25728 -0.15778 -0.091743 0.16179 0.2565 -0.57449 -0.22681 -0.89427 -0.22239 0.52927 0.76028 1.3736 -0.17043 -2.0083 -0.51106 -0.26381 1.5273 0.13254 -0.25583 0.92246 0.22393 0.90556 1.0088 -0.16988 0.74343 -0.35974 -0.077833 -0.35517 -0.79085 -0.44397 0.47479 0.096046 0.0029278 -0.30727 0.28722 0.51917 -0.94567 0.35826 0.82026 -0.57482 -1.0386 -0.14005 -1.8042 0.51904 0.6171 -0.1785 -0.18061 0.057552 -0.49046 -0.17216 0.086454 -0.073225 -0.021868 -0.60123 -0.72079 -0.55905 0.7363 -0.069655

much -0.3384 0.6032 0.61412 -0.05686 -0.37309 -0.061981 -0.40583 -0.11304 0.018956 -0.086392 -0.19767 0.30401 -0.17332 -0.58002 0.25126 -0.83477 -


take -0.27064 0.0051896 0.1497 -0.098242 -0.34941 0.053679 -0.49698 0.65251 -0.34078 -0.23466 0.091924 0.4328 -0.05257 0.25661 -0.073174 -0.31834 0.24386 0.52261 -0.64237 0.3446 0.67449 -0.41091 -0.068067 0.11036 -0.31174 -0.1838 -0.32548 -0.56073 0.46353 -0.38417 -0.66699 0.51162 -0.19582 -0.16548 -0.11617 0.40172 -0.27041 0.12839 -0.24684 -0.086713 -0.51182 -0.11955 0.14814 -0.85205 -0.42312 0.27046 -0.19395 -0.1686 -0.022328 -0.79142 -0.13786 0.084995 -0.19315 1.2555 0.0041198 -2.7418 0.083024 -0.20155 1.8789 0.079497 -0.13951 0.83795 -0.28992 0.035695 0.81729 0.25042 0.06956 0.61749 -0.33027 -0.49086 -0.16137 -0.88796 -0.36941 -0.63618 0.11441 0.087835 -0.7473 -0.1224 -0.50284 -0.28315 0.78754 -0.41615 -0.58013 0.0071536 -1.3391 -0.21096 0.51283 0.4465 -0.0473 -0.47531 -0.101 -0.21284 0.26688 -0.43676 -0.5557 -0.0011168 -0.0016866 -0.23098 0.54587 0.49992

very -0.84136 0.30985 0.05817 -0.1282 -0.57563 -0.090958 -0.14138 0.2938 -0.1028 -0.32226 -0.14369 -0.15385 0.27397 -0.41289 -


according -0.068258 -0.047649 0.49786 -0.40922 -0.0051759 0.012998 0.38682 0.16049 0.13969 0.3777 0.12385 -0.32508 0.76584 -0.33999 -0.032049 -0.58342 0.53179 -0.71871 -0.88889 0.095524 0.17448 -0.4459 0.19492 0.71823 -0.37576 -0.61964 0.44474 -0.5949 -0.26213 0.20811 0.13722 0.033131 -0.23305 -0.0027318 -0.46937 0.47767 0.25512 0.53612 0.045807 -0.28083 -0.415 -0.067618 -0.26842 0.36729 -0.081097 -0.52428 -0.2608 -0.85743 -0.76121 -0.56286 0.7719 -0.03485 0.61521 0.67937 0.074163 -2.0818 -0.26734 -0.17753 1.9872 0.52471 -0.23144 0.41671 0.082092 -0.18117 0.56496 0.10529 -0.24687 0.42676 0.90579 0.10628 -0.26557 0.36568 0.19739 -0.34528 0.22259 0.20374 -0.19925 -0.10147 -1.4524 -0.29982 1.029 0.33442 -0.26938 -0.26667 -0.96846 -0.12894 -0.32834 -0.21234 0.13487 -0.081348 0.81305 -0.40582 -0.19017 -0.006391 -0.31896 0.40277 -0.2682 -0.39942 0.18658 -0.47027

several -0.353 0.010869 -0.72551 -0.043148 0.71976 0.46265 -0.069802 0.28948 -0.29837 -0.081974 0.27666 -0.14696 0.39687 0.28386 

UnicodeDecodeError: 'gbk' codec can't decode byte 0x93 in position 5456: illegal multibyte sequence

### NN

In [ ]:
# Using Neural Networks and Facebook's Fasttext
earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

# NN
def doAddNN(X_train,X_test,pred_train,pred_test):
    for i in range(6):
        X_train['nn_'+str(i)] = pred_train[:,i]
        X_test['nn_'+str(i)] = pred_test[:,i]
    return X_train,X_test

def initNN(nb_words_cnt,max_len):
    model = Sequential()
    model.add(Embedding(nb_words_cnt,32,input_length=max_len))
    model.add(Dropout(0.3))
    model.add(Conv1D(64,
                     5,
                     padding='valid',
                     activation='relu'))
    model.add(Dropout(0.3))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(800, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(labels), activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    return model

def doNN(X_train,X_test,Y_train):
    max_len = 70
    nb_words = 10000
    
    print('Processing text dataset')
    texts_1 = []
    for text in X_train['comment_text']:
        texts_1.append(text)

    print('Found %s texts.' % len(texts_1))
    test_texts_1 = []
    for text in X_test['comment_text']:
        test_texts_1.append(text)
    print('Found %s texts.' % len(test_texts_1))
    
    tokenizer = Tokenizer(num_words=nb_words)
    tokenizer.fit_on_texts(texts_1 + test_texts_1)
    sequences_1 = tokenizer.texts_to_sequences(texts_1)
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    test_sequences_1 = tokenizer.texts_to_sequences(test_texts_1)

    xtrain_pad = pad_sequences(sequences_1, maxlen=max_len)
    xtest_pad = pad_sequences(test_sequences_1, maxlen=max_len)
    del test_sequences_1
    del sequences_1
    nb_words_cnt = min(nb_words, len(word_index)) + 1

    # we need to binarize the labels for the neural net
    
    # ytrain_enc = np_utils.to_categorical(Y_train)
    ytrain_enc = Y_train
        
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([xtrain_pad.shape[0], len(labels)])
    for dev_index, val_index in kf.split(xtrain_pad):
        dev_X, val_X = xtrain_pad[dev_index], xtrain_pad[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initNN(nb_words_cnt,max_len)
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=4, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(xtest_pad)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddNN(X_train,X_test,pred_train,pred_full_test/5)

train_df,test_df = doNN(train_df,test_df,train_y)
print('NN finished...')

### NN Glove

In [ ]:
## NN Glove

def doAddNN_glove(X_train,X_test,pred_train,pred_test):
    for i in range(6):
        X_train['nn_glove_'+str(i)] = pred_train[:,i]
        X_test['nn_glove_'+str(i)] = pred_test[:,i]
    return X_train,X_test

def initNN_glove():
    # create a simple 3 layer sequential neural net
    model = Sequential()

    model.add(Dense(128, input_dim=100, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())

    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())

    model.add(Dense(len(labels)))
    model.add(Activation('softmax'))

    # compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def doNN_glove(X_train,X_test,Y_train,xtrain_glove,xtest_glove):
    # scale the data before any neural net:
    scl = preprocessing.StandardScaler()
    #ytrain_enc = np_utils.to_categorical(Y_train)
    ytrain_enc = Y_train
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    xtrain_glove = scl.fit_transform(xtrain_glove)
    xtest_glove = scl.fit_transform(xtest_glove)
    pred_train = np.zeros([xtrain_glove.shape[0], len(labels)])
    
    for dev_index, val_index in kf.split(xtrain_glove):
        dev_X, val_X = xtrain_glove[dev_index], xtrain_glove[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initNN_glove()
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=10, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(xtest_glove)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddNN_glove(X_train,X_test,pred_train,pred_full_test/5)

train_df,test_df = doNN_glove(train_df,test_df,train_y,glove_vecs_train,glove_vecs_test)
print('NN Glove finished...')

### Fast Text

In [ ]:
# Fast Text

def doAddFastText(X_train,X_test,pred_train,pred_test):
    for i in range(6):
        X_train['ff_'+str(i)] = pred_train[:,i]
        X_test['ff_'+str(i)] = pred_test[:,i]
    return X_train,X_test


def initFastText(embedding_dims,input_dim):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(len(labels), activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

def preprocessFastText(text):
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

def create_docs(df, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df['comment_text']:
        doc = preprocessFastText(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

def doFastText(X_train,X_test,Y_train):
    min_count = 2

    docs = create_docs(X_train)
    tokenizer = Tokenizer(lower=False, filters='')
    tokenizer.fit_on_texts(docs)
    num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

    tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
    tokenizer.fit_on_texts(docs)
    docs = tokenizer.texts_to_sequences(docs)

    maxlen = 300

    docs = pad_sequences(sequences=docs, maxlen=maxlen)
    input_dim = np.max(docs) + 1
    embedding_dims = 20

    # we need to binarize the labels for the neural net
    #ytrain_enc = np_utils.to_categorical(Y_train)
    ytrain_enc = Y_train

    docs_test = create_docs(X_test)
    docs_test = tokenizer.texts_to_sequences(docs_test)
    docs_test = pad_sequences(sequences=docs_test, maxlen=maxlen)
    xtrain_pad = docs
    xtest_pad = docs_test
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([xtrain_pad.shape[0], len(labels)])
    for dev_index, val_index in kf.split(xtrain_pad):
        dev_X, val_X = xtrain_pad[dev_index], xtrain_pad[val_index]
        dev_y, val_y = ytrain_enc[dev_index], ytrain_enc[val_index]
        model = initFastText(embedding_dims,input_dim)
        model.fit(dev_X, y=dev_y, batch_size=32, epochs=25, verbose=1,validation_data=(val_X, val_y),callbacks=[earlyStopping])
        pred_val_y = model.predict(val_X)
        pred_test_y = model.predict(docs_test)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
    return doAddFastText(X_train,X_test,pred_train,pred_full_test/5)

train_df,test_df = doFastText(train_df,test_df,train_y)
print('FastText finished...')

In [ ]:
cols_to_drop = ['comment_text', 'split']
train_X = train_df.drop(cols_to_drop, axis=1)
test_X = test_df.drop(cols_to_drop, axis=1)

## MNB & SVD

In [ ]:
### Fit transform the tfidf vectorizer ###
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
full_tfidf = tfidf_vec.fit_transform(train_df['comment_text'].values.tolist() + test_df['comment_text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['comment_text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['comment_text'].values.tolist())

def runMNB(train_X, train_y, test_X, test_y, test_X2):
    model = naive_bayes.MultinomialNB()
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)
    pred_test_y2 = model.predict_proba(test_X2)

    return pred_test_y, pred_test_y2, model

cv_scores = []
pred_full_test = np.zeros([test_df.shape[0], len(labels)])
pred_train = np.zeros([train_df.shape[0], len(labels)])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    dev_X, dev_y
    for i, j in enumerate(labels):
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y[:,i], val_X, val_y[:,i], test_tfidf)
        pred_test_y[:, 0]
        pred_full_test[:, i] = pred_full_test[:,i] + pred_test_y[:,0] # FIXME
        pred_train[val_index,i] = pred_val_y[:,0]
        cv_scores.append(metrics.log_loss(val_y[:,i], pred_val_y[:,0]))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

In [ ]:
n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_word_'+str(i) for i in range(n_comp)]
train_df = pd.concat([train_df, train_svd], axis=1)
test_df = pd.concat([test_df, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

In [ ]:
### Fit transform the count vectorizer ###
tfidf_vec = CountVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit(train_df['comment_text'].values.tolist() + test_df['comment_text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['comment_text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['comment_text'].values.tolist())

In [ ]:
cv_scores = []
pred_full_test = np.zeros([test_df.shape[0], len(labels)])
pred_train = np.zeros([train_df.shape[0], len(labels)])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    dev_X, dev_y
    for i, j in enumerate(labels):
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y[:,i], val_X, val_y[:,i], test_tfidf)
        pred_test_y[:, 0]
        pred_full_test[:, i] = pred_full_test[:,i] + pred_test_y[:,0] # FIXME
        pred_train[val_index,i] = pred_val_y[:,0]
        cv_scores.append(metrics.log_loss(val_y[:,i], pred_val_y[:,0]))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.


# add the predictions as new features #
for i in range(6):
    train_df["nb_cvec_"+str(i)] = pred_train[:,i]
    test_df["nb_cvec_"+str(i)] = pred_full_test[:,i]
print("Naive Bayesian Count Vector finished...")

In [ ]:
### Fit transform the tfidf vectorizer ###
tfidf_vec = CountVectorizer(ngram_range=(1,7), analyzer='char')
tfidf_vec.fit(train_df['comment_text'].values.tolist() + test_df['comment_text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['comment_text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['comment_text'].values.tolist())

In [ ]:
cv_scores = []
pred_full_test = np.zeros([test_df.shape[0], len(labels)])
pred_train = np.zeros([train_df.shape[0], len(labels)])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    dev_X, dev_y
    for i, j in enumerate(labels):
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y[:,i], val_X, val_y[:,i], test_tfidf)
        pred_test_y[:, 0]
        pred_full_test[:, i] = pred_full_test[:,i] + pred_test_y[:,0] # FIXME
        pred_train[val_index,i] = pred_val_y[:,0]
        cv_scores.append(metrics.log_loss(val_y[:,i], pred_val_y[:,0]))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.


# add the predictions as new features #
for i in range(6):
    train_df["nb_cvec_char_"+str(i)] = pred_train[:,i]
    test_df["nb_cvec_char_"+str(i)] = pred_full_test[:,i]
print("Naive Bayersian Count Vector Char finished...")

In [ ]:
### Fit transform the tfidf vectorizer ###
tfidf_vec = TfidfVectorizer(ngram_range=(1,5), analyzer='char')
full_tfidf = tfidf_vec.fit_transform(train_df['comment_text'].values.tolist() + test_df['comment_text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['comment_text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['comment_text'].values.tolist())

In [ ]:
cv_scores = []
pred_full_test = np.zeros([test_df.shape[0], len(labels)])
pred_train = np.zeros([train_df.shape[0], len(labels)])
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)

for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_tfidf[dev_index], train_tfidf[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    dev_X, dev_y
    for i, j in enumerate(labels):
        pred_val_y, pred_test_y, model = runMNB(dev_X, dev_y[:,i], val_X, val_y[:,i], test_tfidf)
        pred_test_y[:, 0]
        pred_full_test[:, i] = pred_full_test[:,i] + pred_test_y[:,0] # FIXME
        pred_train[val_index,i] = pred_val_y[:,0]
        cv_scores.append(metrics.log_loss(val_y[:,i], pred_val_y[:,0]))
print("Mean cv score : ", np.mean(cv_scores))
pred_full_test = pred_full_test / 5.

# add the predictions as new features #
for i in range(6):
    train_df["nb_tfidf_char_"+str(i)] = pred_train[:,i]
    test_df["nb_tfidf_char_"+str(i)] = pred_full_test[:,i]
print("Naive Bayersian TFIDF Vector Char finished...")

In [ ]:
n_comp = 20
svd_obj = TruncatedSVD(n_components=n_comp, algorithm='arpack')
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
    
train_svd.columns = ['svd_char_'+str(i) for i in range(n_comp)]
test_svd.columns = ['svd_char_'+str(i) for i in range(n_comp)]
train_df = pd.concat([train_df, train_svd], axis=1)
test_df = pd.concat([test_df, test_svd], axis=1)
del full_tfidf, train_tfidf, test_tfidf, train_svd, test_svd

## XGB

In [ ]:
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 3
    param['silent'] = 1
    param['num_class'] = len(labels)
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model

In [ ]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2017)
cv_scores = []
pred_full_test = 0
pred_train = np.zeros([train_df.shape[0], len(labels)])
for dev_index, val_index in kf.split(train_X):
    dev_X, val_X = train_X.loc[dev_index], train_X.loc[val_index]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, test_X, seed_val=0, colsample=0.7)
    pred_full_test = pred_full_test + pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_scores.append(metrics.log_loss(val_y, pred_val_y))
print("cv scores : ", cv_scores)

pred_full_test /= 5.0

## To CSV

In [ ]:
out_df = pd.DataFrame(pred_full_test)
out_df.columns = labels
out_df.insert(0, 'id', test_id)
out_df.to_csv("../output/result.csv", index=False)

In [ ]:
file_name = '../output/train_df.pkl'
train_df.to_pickle(file_name)  
train_df = pd.read_pickle(file_name)
file_name = '../output/test_df.pkl'
test_df.to_pickle(file_name)
test_df = pd.read_pickle(file_name)